In [1]:
from pytrends.request import TrendReq
import pandas as pd
import os
import os.path
import pandas as pd
import time
import psycopg2 
import psycopg2.extras as extras
import matplotlib.pyplot as plt

In [53]:
def menu():
    print(30 * "-", "MENU", 30 * "-")
    print("1. Lấy dữ liệu trending từ file. ")
    print("2. Xuất báo cáo top 10 trending")
    print("3. Xuất báo cáo search keyword in 2020")
    print("4. Vẽ biểu đồ line chart top 5 trending các từ khóa tìm kiếm nhiều nhất 2020")
    print("5. Vẽ biểu đồ bar chart top 5 trending các từ khóa tìm kiếm nhiều nhất 2019")
    print("6. Thống kê tìm kiếm top trending 5 từ khóa trong 2 năm 2020, 2019")
    print(9 * ".")
    print("99.Thoát")

In [54]:
def checkExitsFile(inputFile):
    #if os.path.isfile("/{file}.{ext}".format(file=inputFile, ext="xlsx")):
    if os.path.isfile(inputFile):
        return True
    else:
        return False



In [55]:
def connect():
    """ Connect to the PostgreSQL database server """
    conn, cur = None, None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(
            host="localhost", port="5432",
            database="GoogleTrend",
            user="postgres",
            password="Qu@ng@nh")
        # create a cursor
        cur = conn.cursor()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error while excuting SQL" + error)

    return conn, cur

In [56]:
def getDataFromGoogleTrend(inputFile,FromDate,ToDate):
    
    strTimeFrame=FromDate+' '+ToDate
    checkFile=checkExitsFile(inputFile)
    if checkFile==True:
        df_raw = pd.read_excel(inputFile, sheet_name='key_word')
        #Thay the cot co ky tu / thanh _
        df = df_raw.rename(columns=lambda b: b.replace("/ ","_"))
        column_list = df.columns.ravel()
        
        df_db=[]
        
        #lay du lieu tu google trend
        pytrend = TrendReq(hl='vi-VN', tz=360, timeout=10)
        for i in column_list:
            dataset = []
            keywords = list(df[i].dropna()) 
            for x in range(0,len(keywords)):
                keyword =[keywords[x]]
                pytrend.build_payload(
                    kw_list=keyword,
                    cat=0,
                    timeframe='2019-01-01 2020-12-31',
                    geo='VN',
                    gprop=''
                )
                data=pytrend.interest_over_time()
                if not data.empty:
                    data = data.drop(labels=['isPartial'],axis='columns')
                    dataset.append(data)
            result=pd.concat(dataset,axis=1)
            
           #Tao dataframe cau truc giong voi DB 
            result.reset_index(level=0, inplace=True)
            df_melt=pd.melt(result, id_vars=["date"],var_name="Keywords", value_name="Value")
            df_melt['Trend_type'] = i   
            #result_final=pd.concat([df_melt],ignore_index=True)
            df_db.append(df_melt)
        df_db_final = pd.concat(df_db)        
     
        return df_db_final
        
    else:
        print('File khong ton tai')
    

        
            

In [60]:
def insertGoogleTrend2DB(fromDate,toDate):
    try:
        print('Đang lấy dữ liệu....')
        df=getDataFromGoogleTrend('keytrends.xlsx',fromDate,toDate)
        #Insert vao DB
        conn, cur = connect()
        #Xoa du lieu
        #truncate table
        sqldelete = "delete from vn_trending where date between '"+fromDate+"' AND '"+toDate+"'"
        cur.execute(sqldelete)
        conn.commit()

        # Creating a list of tupples from the dataframe values
        tpls = [tuple(x) for x in df.to_numpy()]

        # dataframe columns with Comma-separated
        cols = ','.join(list(df.columns))

        # SQL query to execute
        sql = "INSERT INTO %s(%s) VALUES %%s" % ("vn_trending", cols)

        extras.execute_values(cur, sql, tpls)
        conn.commit()

        cur.execute("select count(*) from vn_trending where date between '"+fromDate+"' AND '"+toDate+"'")
        row_num = cur.fetchall()

        print("Data inserted %s successfully.." % row_num[0][0])
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()        

In [78]:
def top_10_trending():
    try:
        conn, cur = connect()
        sql_top_10_trending = """
                        with value_month as (
                            select keywords,date_part('month',date) || '-' || date_part('year',date) as Thang, sum(value) as giatri_thang 
                            from vn_trending
                            group by keywords,date_part('month',date) || '-' || date_part('year',date)
                        ), value_month_keyword as (
                        select a.*, b.thang 
                        from (
                            select keywords, max(giatri_thang) as max_giatri_thang
                            from value_month a
                            group by keywords
                        ) a
                        left join value_month b on a.keywords=b.keywords and a.max_giatri_thang=b.giatri_thang
                        ) 
                        select a.keywords, sum(a.value) as GiaTri, b.thang, b.max_giatri_thang  
                        from vn_trending a
                        left join value_month_keyword b on a.keywords = b.keywords
                        group by a.keywords,b.thang, b.max_giatri_thang  
                        order by sum(a.value) desc
                        limit 10
                        """
        cur.execute(sql_top_10_trending)
        # Fetch all the records
        tuples = cur.fetchall()
        # Preparing list of columns for dataframe
        df_top = pd.DataFrame(tuples,columns=['keyword','Số lần tìm kiếm','Tháng tìm kiếm nhiều nhất','Số lần của tháng'])
        df_top.index += 1
        df_top.index.name='STT'
        df_top.to_excel('vn_trending_result.xls')
        print('Lưu kết quả ra file vn_trending_result.xls thành công')
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        
    finally:
        if conn is not None:
            conn.close()    

In [99]:
def search_keyword(year):
    try:
        writer1=pd.ExcelWriter('vn_trending_search_keyword_2020.xlsx', engine='xlsxwriter')
        #workbook = xlsxwriter.Workbook('merge1.xlsx')
        conn, cur = connect()
        sql_distint ="select distinct trend_type from vn_trending"
        cur.execute(sql_distint)
        lst_trend_type = cur.fetchall()
        sql_distint=""
        for i in range(len(lst_trend_type)):
            sql_distint="""
                    select * 
                    from crosstab(
                            'select keywords,date_part(''month'',date) as Thang, sum(value) as giatri
                            from vn_trending
                            where date_part(''year'',date)=''"""+year+"""'' and trend_type=''"""+lst_trend_type[i][0]+"""''
                            group by keywords,date_part(''month'',date) ORDER BY 1',
                            'Select distinct date_part(''month'',date) as Thang from vn_trending where trend_type=''songs'' ORDER BY 1 '
                            ) as ct (
                              keywords text,
                              "Thang 1" bigint,
                              "Thang 2" bigint,
                              "Thang 3" bigint,
                              "Thang 4" bigint,
                              "Thang 5" bigint,
                              "Thang 6" bigint,
                              "Thang 7" bigint,
                              "Thang 8" bigint,
                              "Thang 9" bigint,
                              "Thang 10" bigint,
                              "Thang 11" bigint,
                              "Thang 12" bigint
                            );
                    """
            cur.execute(sql_distint)
            rows = cur.fetchall()
            columns_names = [ x[0] for x in cur.description]
            df1 = pd.DataFrame(rows,columns=columns_names)
            df1.index += 1
            df1.index.name='STT'
            df1.to_excel(writer1,sheet_name=lst_trend_type[i][0],startrow=2)
            
            #Dăt tiêu đề cho các file excel
            ws=writer1.sheets[lst_trend_type[i][0]]
            ws.merge_range('A1:N1', 'TỪ KHÓA TÌM KIẾM NHIỀU NHẤT TẠI VIỆT NAM')
            ws.merge_range('A2:N2', 'NĂM '+year)
        writer1.save()  
        print('Lưu kết quả ra file vn_trending_search_keyword_2020.xlsx thành công')
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)

    finally:
        if conn is not None:
            conn.close() 

In [100]:
search_keyword('2020')

Connecting to the PostgreSQL database...
Lưu kết quả ra file vn_trending_search_keyword_2020.xlsx thành công


In [101]:
menu()
choice=input("Enter join choice[1-5]:")
if choice=='1':
    fromDate=input('Từ ngày: ')
    toDate=input('Đến ngày: ')
    insertGoogleTrend2DB(fromDate,toDate)
    print("Lấy dữ liệu thành công")
    menu()
elif choice=='2':    
    top_10_trending()
    menu()
elif choice=='3':    
    search_keyword('2020')
    menu()    
elif choice=='99':
    menu()
elif choice=='5':
    print('Exit')

------------------------------ MENU ------------------------------
1. Lấy dữ liệu trending từ file. 
2. Xuất báo cáo top 10 trending
3. Xuất báo cáo search keyword in 2020
4. Vẽ biểu đồ line chart top 5 trending các từ khóa tìm kiếm nhiều nhất 2020
5. Vẽ biểu đồ bar chart top 5 trending các từ khóa tìm kiếm nhiều nhất 2019
6. Thống kê tìm kiếm top trending 5 từ khóa trong 2 năm 2020, 2019
.........
99.Thoát
Enter join choice[1-5]:3
Connecting to the PostgreSQL database...
Lưu kết quả ra file vn_trending_search_keyword_2020.xlsx thành công
------------------------------ MENU ------------------------------
1. Lấy dữ liệu trending từ file. 
2. Xuất báo cáo top 10 trending
3. Xuất báo cáo search keyword in 2020
4. Vẽ biểu đồ line chart top 5 trending các từ khóa tìm kiếm nhiều nhất 2020
5. Vẽ biểu đồ bar chart top 5 trending các từ khóa tìm kiếm nhiều nhất 2019
6. Thống kê tìm kiếm top trending 5 từ khóa trong 2 năm 2020, 2019
.........
99.Thoát
